- 게시글 불러오기 + 정보들과 함께.

In [ ]:
from urllib.request import urlopen
import requests
import time
from selenium import webdriver

driver = webdriver.Chrome('C:/Users/사용자이름/Downloads/chromedriver_win32 (2)/chromedriver.exe')

# pageList에 자유게시판 링크 수집 - 원하는대로 range()값 수정하기. 
pageList = []

for i in range(50,1000):
    #if i ==0:
        #nowpage = "https://everytime.kr/370441"
        #pageList.append(nowpage)
    #else:
    nowpage = "https://everytime.kr/370441/p/%s"%(i+1)
    pageList.append(nowpage)
        
# 에브리타임 로그인
driver.get(nowpage)
time.sleep(0.1)
driver.find_element_by_xpath("""//*[@id="container"]/form/p[1]/input""").send_keys("아이디") # 입력 버튼 클릭.    
time.sleep(0.1)
driver.find_element_by_xpath("""//*[@id="container"]/form/p[2]/input""").send_keys("비밀번호")
time.sleep(0.1)
driver.find_element_by_xpath("""//*[@id="container"]/form/p[3]/input""").click()

textList = []
informList = []
titleList = []

#링크 들어가서 글, 스크랩수, 좋아요수, 댓글수를 가져온다.
for now_url in pageList:
    count = 0
    for i in range(1,21):
        driver.get(now_url)
        #날짜 확인하기
        if count ==0:
            while True:
                try:
                    print(driver.find_element_by_xpath('//*[@id="container"]/div[2]/article[8]/a/time').text)
                    count = count+1
                    break
                except:
                    pass
        #해당페이지 게시글 순서대로 들어가기
        while True:
            try:
                driver.find_element_by_xpath('//*[@id="container"]/div[2]/article[%s]/a'%i).click()
                break
            except:
                pass
        #제목 가져오기
        while True:
            try:
                title = driver.find_element_by_xpath('//*[@id="container"]/div[2]/article/a/h2').text
                titleList.append(title)
                break
            except:
                pass        
        #글 가져오기
        while True:
            try:
                text = driver.find_element_by_xpath('//*[@id="container"]/div[2]/article/a/p').text
                textList.append(text)
        #정보 가져오기
                inform = driver.find_element_by_xpath('//*[@id="container"]/div[2]/article/a/ul[2]').text
                informList.append(inform)
                break
            except:
                pass
        textList.append("^")
driver.quit()

#수집한 글 전처리
import re
newList = []

for i in range(len(textList)):
    text = textList[i]
    parse =re.sub('\n', ' ', text)
    print(parse)
    newList.append(parse)
print(newList)

print(titleList)

#글 정리해서 리스트에 저장
mainText = []
totalText = []

for i in range(len(newList)):
    beforeText = newList[i]
    if beforeText != "^":
        mainText.append(beforeText)

#제목과 본문 합치기
for i in range(len(titleList)):
    totalText.append(titleList[i]+" "+mainText[i])
    
print(totalText)

#글자수도 저장
textCount = []

for i in range(len(totalText)):
    textCount.append(len(totalText[i]))
print(textCount)

#정보 리스트에 저장
goodList = []
commentList = []
scrapList = []
mainInform = []

for i in range(len(informList)):
    beforeText = informList[i]
    afterText = beforeText.split("\n")
    mainInform.append(afterText)

for i in range(len(mainInform)):
    if len(mainInform[i])==4:
        del mainInform[i][0]
    goodList.append(mainInform[i][0])
    commentList.append(mainInform[i][1])
    scrapList.append(mainInform[i][2])
    
print("좋아요 : ",goodList)
print("댓글 : ",commentList)
print("스크랩 : ",scrapList)

#리스트를 데이터프레임으로 바꾸기.
totalList = []
import pandas as pd
from pandas import DataFrame
totalList.append(totalText)
totalList.append(textCount)
totalList.append(goodList)
totalList.append(commentList)
totalList.append(scrapList)
print(totalList)

df = DataFrame (totalList).transpose()
df.columns = ['본문',"글자수",'좋아요','댓글수','스크랩수']

df.to_csv('everytimeData.csv',encoding='utf-8-sig')